In [1]:
from sklearn.model_selection import train_test_split
import os
import pandas as pd

# Read the file and divide the sentences into words
with open('generated_data/merged_new.txt', 'r') as f:
    # Read text and write it in data value
    data = f.read()
    # Split words with space
    words = data.split()

sentences = []
current_sentence = []

# Create suggestions for the points found. No other characters are found in the generated data
for word in words:
    current_sentence.append(word)
    if word.endswith('.'):
        sentences.append(current_sentence)
        current_sentence = []

# Delete one-word sentences.
for sent in sentences:
    if len(sent) <= 1:
        sentences.remove(sent)

In [2]:
result_list = []
current_id = 0

# Cycle to find the contents between the two pluses "++".
# Create a list of ner tags and generate data for further loading into json files.
for sentence in sentences:
    ner_tags = []
    updated_sentence = []
    inside_names = 0
    # Take one sentence and detect two pluses inside
    for word in sentence:
        # Copy word? for avoiding data loses
        updated_word = word

        # # B (Begin), I (Inside) или O (Outside).
        # if word.startswith('++'):
        #     updated_word = word.replace('++', '').strip()
        #     ner_tags.append('B-geo')
        # elif word.endswith('++'):
        #     updated_word = word.replace('++', '').strip()
        #     ner_tags.append('I-geo')
        # else:
        #     ner_tags.append('O')

        if word.startswith('++', 0, 2):
            if word.count('+') == 4 and word.endswith("++") is False:
                inside_names = 0
            elif word.endswith("++") is False:
                inside_names += 1
            updated_word = word.replace('++', '').strip()
            ner_tags.append(1)
        elif word.endswith('++') and inside_names >= 1:
            updated_word = word.replace('++', '').strip()
            ner_tags.append(2)
            inside_names = 0
        elif word.count('+') == 2:
            updated_word = word.replace('++', '').strip()
            ner_tags.append(2)
            inside_names = 0
        elif inside_names >= 1:
            ner_tags.append(2)
        else:
            inside_names = 0
            ner_tags.append(0)

        # Add updated word in new list of words
        updated_sentence.append(updated_word)

    current_id += 1

    # Create a dictionary with data
    word_dict = {
        'id': current_id,
        'ner_tags': ner_tags,
        'tokens': updated_sentence
    }
    # Append dict to result list
    result_list.append(word_dict)

In [3]:
# Path to dictionary with training files
TRAIN_READY_DATASET_DIR = "../tokenaized_data"

# Convert list to DataFrame
dataset_df = pd.DataFrame(result_list)

# Split the data into training and evaluation datasets using an 80-20 split ratio
train_df, eval_df = train_test_split(
    dataset_df[['id', 'ner_tags', 'tokens']], shuffle=True, test_size=0.2
)

# Save the training and evaluation datasets as JSON files in the specified directory
train_df.to_json(
    os.path.join(TRAIN_READY_DATASET_DIR, "train_data.json"),
    indent=2,
    orient="table",
    index=None,
)

eval_df.to_json(
    os.path.join(TRAIN_READY_DATASET_DIR, "eval_data.json"),
    indent=2,
    orient="table",
    index=None,
)